## Libraries and settings

Import the required libraries and set the main settings.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
print(os.getcwd())

/workspaces/data_analytics_project/notebooks


# Weather & Bicycle Usage – Data Preprocessing

This notebook cleans, merges, and prepares the collected weather and bicycle traffic data for Zurich for further analysis.

In [2]:
# Load hourly weather data with all features
weather_df = pd.read_csv("../data/weather_zurich_2025.csv")
weather_df['time'] = pd.to_datetime(weather_df['time'])

print("Weather data shape:", weather_df.shape)
print(weather_df.head())
print("\nAvailable columns:", weather_df.columns.tolist())
print("\nData types:")
print(weather_df.dtypes)

Weather data shape: (8760, 5)
                 time  temperature_2m  relative_humidity_2m  wind_speed_10m  \
0 2025-01-01 00:00:00            -1.5                    98             1.0   
1 2025-01-01 01:00:00            -1.9                    99             2.6   
2 2025-01-01 02:00:00            -3.2                   100             2.4   
3 2025-01-01 03:00:00            -3.0                   100             4.7   
4 2025-01-01 04:00:00            -2.8                   100             5.0   

   precipitation  
0            0.0  
1            0.0  
2            0.0  
3            0.0  
4            0.0  

Available columns: ['time', 'temperature_2m', 'relative_humidity_2m', 'wind_speed_10m', 'precipitation']

Data types:
time                    datetime64[ns]
temperature_2m                 float64
relative_humidity_2m             int64
wind_speed_10m                 float64
precipitation                  float64
dtype: object


## Loading datasets

We load both the weather and bicycle counter data from the CSV files generated in the data collection phase.

In [3]:
# Load bicycle counter data
bikes_df = pd.read_csv("../data/2025_verkehrszaehlungen_werte_fussgaenger_velo.csv")
print("Bicycle data shape:", bikes_df.shape)
print(bikes_df.head())
print("\nData types:")
print(bikes_df.dtypes)

Bicycle data shape: (838029, 8)
   FK_STANDORT             DATUM  VELO_IN  VELO_OUT  FUSS_IN  FUSS_OUT  \
0         4241  2025-01-01T00:00      3.0       0.0      NaN       NaN   
1         2989  2025-01-01T00:00      1.0       2.0      NaN       NaN   
2         2991  2025-01-01T00:00     18.0       2.0      NaN       NaN   
3         4255  2025-01-01T00:00      0.0       0.0      NaN       NaN   
4         4242  2025-01-01T00:00      1.0       0.0      NaN       NaN   

       OST     NORD  
0  2682297  1248328  
1  2682278  1248324  
2  2682756  1247323  
3  2682881  1246549  
4  2682337  1248451  

Data types:
FK_STANDORT      int64
DATUM           object
VELO_IN        float64
VELO_OUT       float64
FUSS_IN        float64
FUSS_OUT       float64
OST              int64
NORD             int64
dtype: object


## Cleaning and handling missing values

We convert time columns to datetime format and remove any rows with missing values to ensure data quality.

In [4]:
# Convert date column in bikes_df to datetime
bikes_df["DATUM"] = pd.to_datetime(bikes_df["DATUM"])
print("After datetime conversion:")
print("Bikes DATUM data type:", bikes_df["DATUM"].dtype)

After datetime conversion:
Bikes DATUM data type: datetime64[ns]


In [5]:
# --- Aggregate weather data to daily level ---
# Extract date from time
weather_df['date'] = weather_df['time'].dt.date

# Aggregate weather features to daily level
daily_weather = weather_df.groupby('date').agg({
    'temperature_2m': 'mean',
    'relative_humidity_2m': 'mean',
    'wind_speed_10m': 'mean',
    'precipitation': 'sum'  # Sum for precipitation (total per day)
}).reset_index()

# Rename columns for clarity
daily_weather.rename(columns={
    'temperature_2m': 'temp_mean',
    'relative_humidity_2m': 'humidity_mean',
    'wind_speed_10m': 'wind_speed_mean',
    'precipitation': 'precipitation_sum'
}, inplace=True)

# Convert 'date' back to datetime for merging
daily_weather['date'] = pd.to_datetime(daily_weather['date'])

print('Aggregated daily weather data:')
print(daily_weather.head())
print(f'\nShape: {daily_weather.shape}')

# --- Aggregate bicycle data to daily level ---
# Create a new column with only the date (no time)
bikes_df['DATE_ONLY'] = bikes_df['DATUM'].dt.date

# Sum all bike counts per day (across all locations)
daily_bikes = bikes_df.groupby('DATE_ONLY').agg({
    'VELO_IN': 'sum',
    'VELO_OUT': 'sum',
    'FUSS_IN': 'sum',
    'FUSS_OUT': 'sum'
}).reset_index()

daily_bikes.rename(columns={'DATE_ONLY': 'date'}, inplace=True)
# Convert 'date' back to datetime for merging
daily_bikes['date'] = pd.to_datetime(daily_bikes['date'])

print('\nAggregated daily bicycle data:')
print(daily_bikes.head())
print(f'Shape: {daily_bikes.shape}')

Aggregated daily weather data:
        date  temp_mean  humidity_mean  wind_speed_mean  precipitation_sum
0 2025-01-01   0.650000      86.083333         4.750000                0.0
1 2025-01-02   3.141667      76.958333        12.662500               14.5
2 2025-01-03  -0.308333      87.333333         5.908333                1.4
3 2025-01-04  -1.900000      80.916667         3.183333                0.6
4 2025-01-05   1.845833      93.000000         4.608333               15.0

Shape: (365, 5)

Aggregated daily bicycle data:
        date  VELO_IN  VELO_OUT  FUSS_IN  FUSS_OUT
0 2025-01-01   5094.0    2558.0   1188.0    1099.0
1 2025-01-02   5086.0    2423.0    541.0     448.0
2 2025-01-03   9073.0    4420.0    450.0     404.0
3 2025-01-04   7129.0    3551.0    457.0     388.0
4 2025-01-05   5000.0    2641.0    630.0     565.0
Shape: (359, 5)


In [6]:
# Check for missing values in aggregated data
print("Missing values in daily weather data:")
print(daily_weather.isnull().sum())

print("\nMissing values in daily bicycle data:")
print(daily_bikes.isnull().sum())

Missing values in daily weather data:
date                 0
temp_mean            0
humidity_mean        0
wind_speed_mean      0
precipitation_sum    0
dtype: int64

Missing values in daily bicycle data:
date        0
VELO_IN     0
VELO_OUT    0
FUSS_IN     0
FUSS_OUT    0
dtype: int64


In [7]:
# Drop missing values
daily_weather = daily_weather.dropna()
daily_bikes = daily_bikes.dropna()

print("After dropping missing values:")
print("Daily weather data shape:", daily_weather.shape)
print("Daily bicycle data shape:", daily_bikes.shape)

After dropping missing values:
Daily weather data shape: (365, 5)
Daily bicycle data shape: (359, 5)


In [8]:
# Check for duplicates
print("Duplicates in daily weather data:", daily_weather.duplicated().sum())
print("Duplicates in daily bicycle data:", daily_bikes.duplicated().sum())

# Drop duplicates if any
daily_weather = daily_weather.drop_duplicates()
daily_bikes = daily_bikes.drop_duplicates()

print("\nAfter removing duplicates:")
print("Daily weather data shape:", daily_weather.shape)
print("Daily bicycle data shape:", daily_bikes.shape)

Duplicates in daily weather data: 0
Duplicates in daily bicycle data: 0

After removing duplicates:
Daily weather data shape: (365, 5)
Daily bicycle data shape: (359, 5)


## Merging weather and bicycle datasets

We perform an inner join on the time column to combine both datasets into a single analysis-ready dataset.

In [9]:
# Merge weather and aggregated bicycle data on date
merged_df = pd.merge(daily_weather, daily_bikes, on="date", how="inner")

print("Merged data shape:", merged_df.shape)
print("\nColumns:", merged_df.columns.tolist())
print("\nFirst 5 rows:")
print(merged_df.head())
print("\nData info:")
print(merged_df.info())

Merged data shape: (359, 9)

Columns: ['date', 'temp_mean', 'humidity_mean', 'wind_speed_mean', 'precipitation_sum', 'VELO_IN', 'VELO_OUT', 'FUSS_IN', 'FUSS_OUT']

First 5 rows:
        date  temp_mean  humidity_mean  wind_speed_mean  precipitation_sum  \
0 2025-01-01   0.650000      86.083333         4.750000                0.0   
1 2025-01-02   3.141667      76.958333        12.662500               14.5   
2 2025-01-03  -0.308333      87.333333         5.908333                1.4   
3 2025-01-04  -1.900000      80.916667         3.183333                0.6   
4 2025-01-05   1.845833      93.000000         4.608333               15.0   

   VELO_IN  VELO_OUT  FUSS_IN  FUSS_OUT  
0   5094.0    2558.0   1188.0    1099.0  
1   5086.0    2423.0    541.0     448.0  
2   9073.0    4420.0    450.0     404.0  
3   7129.0    3551.0    457.0     388.0  
4   5000.0    2641.0    630.0     565.0  

Data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359 entries, 0 to 358
Data columns (tot

In [10]:
# Basic statistics
print("Basic statistics:")
print(merged_df.describe())

Basic statistics:
                      date   temp_mean  humidity_mean  wind_speed_mean  \
count                  359  359.000000     359.000000       359.000000   
mean   2025-06-29 00:00:00   10.576346      81.147516         5.490796   
min    2025-01-01 00:00:00   -2.887500      50.583333         1.025000   
25%    2025-03-31 12:00:00    4.533333      74.895833         2.987500   
50%    2025-06-29 00:00:00    9.954167      82.333333         4.662500   
75%    2025-09-26 12:00:00   16.435417      88.416667         6.945833   
max    2025-12-25 00:00:00   26.787500      98.708333        21.987500   
std                    NaN    7.308711      10.197086         3.368768   

       precipitation_sum       VELO_IN      VELO_OUT      FUSS_IN     FUSS_OUT  
count         359.000000    359.000000    359.000000   359.000000   359.000000  
mean            3.530641  23350.487465  11717.409471  1220.885794   941.256267  
min             0.000000   1373.000000   1241.000000     0.000000     0.

In [11]:
# Save merged data
merged_df.to_csv("../data/merged_weather_bikes.csv", index=False)
print("Merged data saved to ../data/merged_weather_bikes.csv")

Merged data saved to ../data/merged_weather_bikes.csv


## Saving cleaned dataset

The merged and cleaned dataset is saved as a CSV file for use in exploratory data analysis.

## Conclusions

In this notebook, we successfully cleaned and merged the weather and bicycle counter datasets for Zurich. We aggregated hourly weather data (temperature, humidity, wind speed, precipitation) to daily means/sums and combined them with daily bicycle counts. Missing values were removed and duplicates were eliminated. The final merged dataset contains daily observations with comprehensive weather features (temp_mean, humidity_mean, wind_speed_mean, precipitation_sum) and bicycle counts. The cleaned dataset is now ready for exploratory data analysis and modeling.

### Jupyter notebook --footer info--

In [12]:
import os
import platform
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.8.0-1030-azure
Datetime: 2026-01-02 13:50:38
Python Version: 3.12.1
-----------------------------------
